In [1]:
import pandas as pd
import numpy as np

In [2]:
#sizes = {'real': {'train': 443757, 'val': 214354}, 'abstract': {'train': 60000, 'val': 30000}}

In [3]:
def get_summary(scenario, _set = 'val'):
    details_train = pd.read_csv('./data/predictions_details_{}_train'.format(scenario), header=None)
    details_train['_set'] = 'train'
    details_val = pd.read_csv('./data/predictions_details_{}_val'.format(scenario), header=None)
    details_val['_set'] = 'val'
    details = pd.concat([details_train, details_val])
    details[2] = details[2].astype(int)
    details.sort_values(by=2, inplace=True)

    prediction = pd.read_csv('./concatenated_data/{}_prediction.csv'.format(scenario))
    prediction.rename(columns={'2049':'q_type', '2048':'q_id'}, inplace=True)
    prediction['base_acc'] = (prediction.base_predicted == prediction.base_target).astype(int)
    prediction['_set'] = details.set_index(2)[['_set']].loc[prediction['q_id'].astype(int)].values
    prediction['my_acc'] = (prediction.my_predicted == prediction.my_target).astype(int)
    prediction['base_acc'] = (prediction.base_predicted == prediction.base_target).astype(int)
    sset_prediction = prediction[prediction._set == _set]
    accs = (sset_prediction[['my_acc','base_acc']].sum()/sset_prediction.shape[0]).values
    print('acc', accs, accs[0] > accs[1])
    accurracies = sset_prediction[['q_type','my_acc','base_acc',
                              'sum_top_down', 'sum_bottom_up']].groupby(['q_type']).sum().reset_index()
    accurracies = accurracies.set_index('q_type')
    
    
    new_summary = pd.read_csv('./concatenated_data/queries_summary_{}.csv'.format(scenario))
    del new_summary['count_top_down']
    new_summary.rename(columns={'2049':'_q_type'}, inplace=True)
    
    
    new_summary = new_summary[new_summary._q_type.isin(accurracies.index.values)]
    new_summary['q_type'] = accurracies.loc[new_summary._q_type.values].index.values
    new_summary['my_acc'] = accurracies.loc[new_summary._q_type.values].my_acc.values
    new_summary['base_acc'] =  accurracies.loc[new_summary._q_type.values].base_acc.values
    val_count = pd.DataFrame(sset_prediction.q_type.value_counts())
    new_summary['_count'] = val_count.loc[new_summary._q_type.values].values
    new_summary['above_median'] = new_summary['_count'] > float(new_summary['_count'].median())
    new_summary['sum_top_down'] = accurracies.loc[new_summary._q_type.values].sum_top_down.values
    new_summary['sum_bottom_up'] =  accurracies.loc[new_summary._q_type.values].sum_bottom_up.values
    new_summary['winner'] = np.array(['top_down',
                                      'bottom_up'])[(new_summary.sum_bottom_up 
                                       > new_summary.sum_top_down).astype(int)]
    _sum = new_summary[['my_acc','base_acc']].sum()
    print('acc_sum', _sum.values, _sum[0] > _sum[1])#/accurracies.shape[0]
    assert (new_summary._q_type != new_summary.q_type).sum() == 0
    _sum = (new_summary.sum_top_down + new_summary.sum_bottom_up).values
    new_summary['sum_top_down'] = new_summary['sum_top_down']/_sum
    new_summary['sum_bottom_up'] = new_summary['sum_bottom_up']/_sum
    new_summary['_diff'] = (new_summary.sum_top_down - new_summary.sum_bottom_up)
    del new_summary['diff']
    del new_summary['q_type']
    del new_summary['sum_top_down']

    print(new_summary._count.sum())
    return new_summary

In [38]:
def get_differents(p=1):
    mean_dif = summary._diff.mean()
    std_dif = summary._diff.std()
    data_dif = summary._diff
    data_diff = pd.DataFrame( (data_dif < mean_dif - p * std_dif).astype(int) + 
                             (data_dif > mean_dif + p * std_dif).astype(int) > 0 )
    significantly_different = summary.loc[data_diff[data_diff._diff].index]
    print(data_diff.shape)
    different = significantly_different[significantly_different._count>2].sort_values(by='winner')
    print(np.round(float(different.shape[0]) / data_diff.shape[0] * 100, 3),'% query_types')

    print(np.round(float(different._count.sum())/ summary._count.sum() * 100, 3),'% queries')

    vals = different[['my_acc','base_acc']].sum().values
    different['my_minus_base'] = different.my_acc - different.base_acc
    print(vals, vals[0]>vals[1])
    print(different.shape[0],'len')
    return different

In [54]:
scenario = 'real'
summary = get_summary(scenario)

('acc', array([ 0.54500966,  0.54460845]), True)
('acc_sum', array([116825, 116739]), True)
214354


In [60]:
#p = 1.31
#p = 1.38
#p = 1
different = get_differents(1)

(317, 1)
(12.303000000000001, '% query_types')
(11.653, '% queries')
(array([15996, 15916]), True)
(39, 'len')


In [61]:
different

,_q_type,sum_bottom_up,count_bottom_up,_count,over_median,winner,my_acc,base_acc,above_median,_diff,my_minus_base
40,boy or,0.517917,0.5,3,False,bottom_up,2,2,False,-0.035834,0
501,who rides,0.524430,0.5,7,True,bottom_up,5,4,False,-0.048860,1
485,who many,0.514453,0.5,18,True,bottom_up,4,3,True,-0.028907,1
422,where might,0.521554,0.5,12,True,bottom_up,5,5,True,-0.043107,0
546,will be,0.514475,0.5,39,True,bottom_up,17,18,True,-0.028951,-1
410,when did,0.518310,0.5,7,True,bottom_up,0,0,False,-0.036619,0
397,what is the color of the,0.518357,0.5,826,True,bottom_up,583,588,True,-0.036714,-5
394,what is in the,0.517037,0.5,1748,True,bottom_up,731,755,True,-0.034074,-24
391,what color is the,0.514800,0.5,14073,True,bottom_up,9774,9699,True,-0.029601,75
389,what color are the,0.517062,0.5,3221,True,bottom_up,2162,2166,True,-0.034123,-4
